In [ ]:
library(autoFRK)
library(sp)
library(mgcv)
library(autoFRK)
library(ggplot2)
library(colorRamps)
library(grid)
library(tidyverse)
library(RColorBrewer)
library(caret)

In [ ]:
a1_train = read.csv('../input/2022-kaust-ss-competition-1a/1a_1_train.csv', row.names = 1)
a1_test = read.csv('../input/2022-kaust-ss-competition-1a/1a_1_test.csv', row.names = 1)
a2_train = read.csv('../input/2022-kaust-ss-competition-1a/1a_2_train.csv', row.names = 1)
a2_test = read.csv('../input/2022-kaust-ss-competition-1a/1a_2_test.csv', row.names = 1)
sample_submission_1 = read.csv('../input/2022-kaust-ss-competition-1a/1a_solution_sample1.csv')
# submission <- read.csv('a1yt_a2gpgp100.csv')

# detrain main impluse response

In [ ]:
main_df <- data.frame(a=a1_train$x+a1_train$y,z=a1_train$z)
main_gam <- gam(z~s(a,k=1000,bs="cr"),data=main_df)

##### detrain main impluse response
main_df %>% ggplot() +
  aes(x=a, y=main_gam$fitted.values) + 
  geom_point(color='black') + 
  aes(x=a, y=z) + 
  geom_line(color='blue', lwd=2) + 
  theme_minimal()

In [ ]:
myPalette <- colorRampPalette(rev(brewer.pal(11, "Spectral"))) # color palette

main_pred <- predict(main_gam, data.frame(a=main_df$a))
demain_z <- main_df$z - main_pred
main_sc <- scale_colour_gradientn(colours = myPalette(100), 
                             limits=c(min(demain_z), max(demain_z)))
ggplot(a1_train, aes(x = x, y = y, colour = demain_z)) +
  geom_point() + main_sc

# detrain second impluse response

In [ ]:
##### detrain second impluse response
sec_df <- data.frame(x=a1_train$x, y=a1_train$y, z = demain_z)

ctp = 1600

x_seq = (c(seq(0, 1, length.out = sqrt(ctp))))
y_seq = (c(seq(0, 1, length.out = sqrt(ctp))))

x_seq_2 = (c(seq(0, 0.125, length.out = sqrt(400))))
y_seq_2 = (c(seq(0, 0.125, length.out = sqrt(400))))

mt = expand.grid(c(x_seq, x_seq_2), c(y_seq, y_seq_2))

mbs = mrts(mt, k = 2000, x = sec_df[,c(1,2)])

In [ ]:
ctp = 1600

x_seq = (c(seq(0, 1, length.out = sqrt(ctp))))
y_seq = (c(seq(0, 1, length.out = sqrt(ctp))))

x_seq_2 = (c(seq(0, 0.125, length.out = sqrt(400))))
y_seq_2 = (c(seq(0, 0.125, length.out = sqrt(400))))

mt = expand.grid(c(x_seq, x_seq_2), c(y_seq, y_seq_2))

plot(mt)

In [ ]:
mbs = cbind(as.matrix(mbs), demain_z)

In [ ]:
write.csv(mbs, 'mbs.csv')

In [ ]:
model = lm(formula = demain_z ~ .,
            data = as.data.frame(mbs))

In [ ]:
sec_pred = predict(model, as.data.frame(mbs))

In [ ]:
cat('Start fitting')
desec_z <- demain_z - sec_pred
sec_sc <- scale_colour_gradientn(colours = myPalette(100), 
                             limits=c(min(desec_z), max(desec_z)))
ggplot(sec_df, aes(x = x, y = y, colour = desec_z)) +
  geom_point() + sec_sc

# Predict Test

In [ ]:
main = predict(main_gam, data.frame(a=a1_test$x+a1_test$y))

In [ ]:
sec_df <- data.frame(x=a1_test$x, y=a1_test$y)

ctp = 1600

x_seq = (c(seq(0, 1, length.out = sqrt(ctp))))
y_seq = (c(seq(0, 1, length.out = sqrt(ctp))))

x_seq_2 = (c(seq(0, 0.125, length.out = sqrt(400))))
y_seq_2 = (c(seq(0, 0.125, length.out = sqrt(400))))

mt = expand.grid(c(x_seq, x_seq_2), c(y_seq, y_seq_2))

mbs = mrts(mt, k = ctp+400, x = sec_df)

In [ ]:
write.csv(mbs, 'mbs_test.csv')

In [ ]:
sec = predict(model, as.data.frame(as.matrix(mbs)))

In [ ]:
submission = read.csv('../input/a1yt-a2gpgp100/a1yt_a2gpgp100_pl.csv')
mean(submission$z1 - (main+sec))
submission$z1 <- main + sec
write.csv(submission, 'submission.csv', row.names=F)